In [ ]:
import os

from slt_datasets.SLTDataset import SLTDataset


DATASET = "RWTH_PHOENIX_2014T"
DATA_DIR = "/mnt/disk3Tb/slt-datasets/"
INPUT_MODE = "pose"
OUTPUT_MODE = "text"
OUTPUT_DIR = "/mnt/disk3Tb/augmented-slt-datasets"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
dataset = SLTDataset(
	data_dir=DATA_DIR + DATASET,
	input_mode=INPUT_MODE,
	output_mode=OUTPUT_MODE,
	split="train"
)

Loaded metadata for dataset: RWTH-PHOENIX-Weather 2014 T: Parallel Corpus of Sign Language Video, Gloss and Translation
Loaded train annotations at /mnt/disk3Tb/slt-datasets/RWTH_PHOENIX_2014T/annotations.csv


Validating files: 100%|██████████| 7096/7096 [00:00<00:00, 23329.23it/s]

Dataset loaded correctly



In [8]:
annotations = dataset.annotations
annotations.head()

,id,text,gloss,signer,split
0,03February_2011_Thursday_heute-600,guten abend liebe zuschauer,GUT ABEND LIEB ZUSCHAUER,Signer08,train
1,20November_2010_Saturday_tagesschau-1833,im bergland fällt zunehmend schnee,BERG DAZU SCHNEE,Signer04,train
2,26April_2010_Monday_heute-3400,und der wind weht auch noch kräftig aus west b...,UNWETTER WEHEN,Signer01,train
3,19November_2011_Saturday_tagesschau-3694,die aussichten von montag bis mittwoch ändert ...,WIE-AUSSEHEN IN-KOMMEND MONTAG BIS MITTWOCH WI...,Signer04,train
4,28March_2011_Monday_tagesschau-2288,über dem bergland können sich einzelne quellwo...,BERG HOCH KOENNEN QUELL WOLKE NAH REGION KOENN...,Signer01,train


In [ ]:
# RWTH-Phoenix data augmentation (OpenAI) – compatible con Jupyter

# Auto-install nest_asyncio if missing
try:
    import nest_asyncio
except ImportError:
    import sys, subprocess, pkg_resources
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "nest_asyncio"])
    import nest_asyncio

import asyncio, os
from typing import List
import pandas as pd
import openai
import nest_asyncio; nest_asyncio.apply()          # <-- permite reusar el event loop

# ---------- CONFIGURATION ----------
OPENAI_API_KEY  = ""
MODEL_NAME      = "gpt-4o-mini"                    # ajusta según tu cuota
AUG_FACTOR      = 2     # paraphrases per original
BATCH_SIZE      = 8     # concurrent requests
TEMPERATURE     = 0.8
CSV_IN          = "train.tsv"                      # archivo original
CSV_OUT         = "train_aug.csv"
# -----------------------------------

client = openai.AsyncOpenAI(api_key=OPENAI_API_KEY)

SYSTEM_PROMPT = (
    "You are a helpful assistant rewriting German sentences.\n"
    "For every user input, produce {n} paraphrases that:\n"
    "• Preserve the exact meaning, tense and register.\n"
    "• Reuse at least 70% of the original words (higher is better).\n"
    "• Vary mainly through word order or minor synonym substitutions.\n"
    "• Keep length within ±3 tokens of the original.\n"
    "• Do NOT add or omit information.\n"
    "Return only the paraphrases, one per line, no numbering."
)

async def paraphrase_batch(sentences: List[str], n: int) -> List[List[str]]:
    tasks = []
    for s in sentences:
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT.format(n=n)},
            {"role": "user",   "content": s},
        ]
        tasks.append(
            client.chat.completions.create(
                model=MODEL_NAME,
                messages=messages,
                temperature=TEMPERATURE,
                max_tokens=128,
                n=1,
                stream=False,
            )
        )
    responses = await asyncio.gather(*tasks)
    result = []
    for r in responses:
        text = r.choices[0].message.content.strip()
        variants = [v.strip() for v in text.splitlines() if v.strip()]
        while len(variants) < n:
            variants.append(variants[-1])
        result.append(variants[:n])
    return result

async def augment_dataframe(df: pd.DataFrame, n: int) -> pd.DataFrame:
    new_rows = []
    for start in range(0, len(df), BATCH_SIZE):
        batch = df.iloc[start : start + BATCH_SIZE]
        variants_list = await paraphrase_batch(batch["text"].tolist(), n)
        for (_, row), variants in zip(batch.iterrows(), variants_list):
            for v in variants:
                new_rows.append(
                    {"id": row["id"], "text": v, "gloss": row["gloss"],
                     "signer": row["signer"], "split": row["split"]}
                )
    return pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

# -------- RUN ----------
df_original = annotations
loop = asyncio.get_event_loop()
df_augmented = loop.run_until_complete(augment_dataframe(df_original, AUG_FACTOR))

/home/pdalbianco/.local/lib/python3.10/site-packages/anyio/streams/tls.py:227: RuntimeWarning: coroutine 'augment_dataframe' was never awaited
  TLSAttribute.peer_certificate: lambda: self._ssl_object.getpeercert(False),


Augmented dataset saved to train_aug.tsv with 21288 rows.


In [26]:
dataset_val = SLTDataset(
	data_dir=DATA_DIR + DATASET,
	input_mode=INPUT_MODE,
	output_mode=OUTPUT_MODE,
	split="val"
)
dataset_test = SLTDataset(
	data_dir=DATA_DIR + DATASET,
	input_mode=INPUT_MODE,
	output_mode=OUTPUT_MODE,
	split="test"
)

df_final = pd.concat([df_augmented, dataset_val.annotations, dataset_test.annotations])
print(len(df_augmented))
print(len(dataset_val.annotations))
print(len(dataset_test.annotations))
print(len(df_final))

Loaded metadata for dataset: RWTH-PHOENIX-Weather 2014 T: Parallel Corpus of Sign Language Video, Gloss and Translation
Loaded val annotations at /mnt/disk3Tb/slt-datasets/RWTH_PHOENIX_2014T/annotations.csv


Validating files: 100%|██████████| 519/519 [00:00<00:00, 196981.61it/s]


Dataset loaded correctly

Loaded metadata for dataset: RWTH-PHOENIX-Weather 2014 T: Parallel Corpus of Sign Language Video, Gloss and Translation
Loaded test annotations at /mnt/disk3Tb/slt-datasets/RWTH_PHOENIX_2014T/annotations.csv


Validating files: 100%|██████████| 642/642 [00:00<00:00, 186194.38it/s]

Dataset loaded correctly

21288
519
642
22449


In [27]:
CSV_OUT         = "train_aug.csv"

df_final.to_csv(CSV_OUT, index=False)
print(f"Augmented dataset saved to {CSV_OUT} with {len(df_final)} rows.")

Augmented dataset saved to train_aug.csv with 22449 rows.


In [15]:
# set max width of pandas df to none
pd.set_option('display.max_colwidth', None)

# get for the same id all the different texts
df_augmented.groupby("id").agg({"text": "unique"})

,text
id,
01April_2010_Thursday_heute-6694,"[liebe zuschauer guten abend, guten abend, liebe zuschauer, guten abend, geschätzte zuschauer]"
01April_2010_Thursday_heute-6695,"[heftiger wintereinbruch gestern in nordirland schottland, Gestern gab es einen heftigen Wintereinbruch in Nordirland und Schottland., In Nordirland und Schottland ereignete sich gestern ein heftiger Wintereinbruch.]"
01April_2010_Thursday_heute-6696,"[schwere überschwemmungen in den usa, schwere Überschwemmungen in den USA, in den USA gibt es schwere Überschwemmungen]"
01April_2010_Thursday_heute-6700,"[weiterhin warm am östlichen mittelmeer und auch richtung westliches mittelmeer ganz west und nordeuropa bleibt kühl, Am östlichen Mittelmeer bleibt es weiterhin warm, während es in Richtung westliches Mittelmeer und ganz West- sowie Nordeuropa kühl bleibt., Es bleibt weiterhin warm am östlichen Mittelmeer, während es in Richtung westliches Mittelmeer sowie im gesamten West- und Nordeuropa kühl ist.]"
01April_2010_Thursday_heute-6701,"[und sehr kühl wird auch die kommende nacht, und die kommende Nacht wird auch sehr kühl, und auch die nächste Nacht wird sehr kühl sein]"
...,...
31October_2009_Saturday_tagesschau-7146,"[im übrigen land schwacher bis mäßiger später kräftig auffrischender wind aus süd bis südost, Im übrigen Land weht ein schwacher bis mäßiger, später kräftig auffrischender Wind aus südlichen bis südöstlichen Richtungen., Im übrigen Land gibt es schwachen bis mäßigen Wind, der später kräftig aus süd bis südost auffrischt.]"
31October_2009_Saturday_tagesschau-7148,"[morgen an der spree frische sechs im südwesten milde siebzehn grad, Morgen an der Spree milde siebzehn Grad und frische sechs im Südwesten., Milde siebzehn Grad morgen an der Spree, frische sechs im Südwesten.]"
31October_2009_Saturday_tagesschau-7149,"[am montag gibt es in der südosthälfte zum teil ergiebige regenfälle sonst einzelne schauer und etwas sonne, Am Montag gibt es in der südostlichen Hälfte teils ergiebige Regenfälle, ansonsten einzelne Schauer und etwas Sonne., In der südostlichen Hälfte gibt es am Montag zum Teil ergiebige Regenfälle, sonst auch einzelne Schauer und etwas Sonne.]"
